In [2]:
import pandas as pd
import re
from collections import defaultdict

In [15]:
df_en=pd.read_json('OneDrive_1_22-1-2026/cc100_en.jsonl',lines=True)
df_mn=pd.read_json('OneDrive_1_22-1-2026/cc100_mn.jsonl',lines=True)

In [17]:
train_size=int(df_en.shape[0]*0.8)
test_size=int(df_en.shape[0]*0.1)
val_size=int(df_en.shape[0]*0.1)
df_en_train=df_en["text"][:train_size]
df_en_test=df_en["text"][train_size:train_size+test_size]
df_en_val=df_en["text"][train_size+test_size:train_size+test_size+val_size]

In [4]:
# Function to remove extra white spaces
def white_space_remover(text):
    return " ".join(text.strip().split())   
df_en["text"]=df_en["text"].apply(white_space_remover)
df_mn["text"]=df_mn["text"].apply(white_space_remover)

In [5]:
df_en.head()

,text
0,Belmont Estate is on the market for $63 millio...
1,“Within the city we’ve had homes that have sol...
2,"The three-storey home has five bedrooms, twelv..."
3,Surrounding the property is a Versailles-inspi...
4,"According to Frosch, the listing has received ..."


In [6]:
df_mn.head()

,text
0,Түмэнбаярын Бум-Эрдэнэ: “Дотроосоо” - 2 (Өгүүл...
1,Маш их таалагдлаа
2,"Хэцүү юмдаа, хүний амьдрал. Хажуудаа жаргал нь..."
3,хаа очиж аймшгийн байсан амьдрал ингэж нэг хэв...
4,Бум ахын өгүүллэгүүд яг л хуучны кино шиг сана...


In [7]:
# Tokenizer function using white space
def white_space_tokenizer(text):
    return text.split()

In [8]:
lis=[]
for i in range(len(df_en)):
    lis.append(white_space_tokenizer(df_en["text"].iloc[i]))

In [9]:
# Regex Tokenizer function
def regex_tokenizer(text):
    # First, normalize number formatting: replace spaces between digits with nothing
    # This converts "22 000" or "1 28" patterns to "22000" or "128"
    text = re.sub(r'(\d)\s+(?=\d)', r'\1', text)
    
    # Pattern explanation:
    # \d{1,3}(?:,\d{3})+(?:\.\d+)? - matches numbers with commas (e.g., 22,000)
    # \b\w+(?:'\w+)? - matches words with optional apostrophes (for contractions like "don't")
    # \d+(?:\.\d+)? - matches numbers including decimals
    # (?<=[a-zA-Z])[.!?;:,] - matches punctuation that follows letters
    # [-–—] - matches various dash types
    pattern = r"\d{1,3}(?:,\d{3})+(?:\.\d+)?|\b\w+(?:'\w+)?\b|\d+(?:\.\d+)?|(?<=[a-zA-Z])[.!?;:,]|[-–—]"
    
    tokens = re.findall(pattern, text)
    return tokens

In [ ]:
def train_bpe(corpus, num_merges=200):
    vocab = defaultdict(int)

    # Build initial vocab
    for sentence in corpus:
        for word in sentence.split():
            vocab[tuple(word) + ("</w>",)] += 1

    merges = []

    for _ in range(num_merges):
        pair_freq = defaultdict(int)

        # Count adjacent pairs
        for tokens, freq in vocab.items():
            for i in range(len(tokens) - 1):
                pair_freq[(tokens[i], tokens[i + 1])] += freq

        if not pair_freq:
            break

        best_pair = max(pair_freq, key=pair_freq.get)

        # Early stopping
        if pair_freq[best_pair] < 2:
            break

        merges.append(best_pair)

        new_vocab = defaultdict(int)

        for tokens, freq in vocab.items():
            new_tokens = []
            i = 0
            while i < len(tokens):
                if i < len(tokens) - 1 and tokens[i] == best_pair[0] and tokens[i + 1] == best_pair[1]:
                    new_tokens.append(tokens[i] + tokens[i + 1])
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1

            new_vocab[tuple(new_tokens)] += freq

        vocab = new_vocab

    return merges

In [ ]:
def apply_bpe(text, merges):
    """
    text: input sentence (string)
    merges: list of BPE merge rules learned from training
    returns: list of BPE tokens
    """

    output_tokens = []

    words = text.split()

    for word in words:
        tokens = list(word) + ['</w>']

        for merge in merges:
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == merge:
                    new_tokens.append(tokens[i] + tokens[i + 1])
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens

        # Remove end-of-word marker
        if tokens[-1] == '</w>':
            tokens = tokens[:-1]

        output_tokens.extend(tokens)

    return output_tokens


In [30]:
train=train_bpe(df_en_train, num_merges=500)

In [36]:

apply_bpe("how are you",train)

['how</w>', 'are</w>', 'you</w>']